In [1]:
!pip install beautifulsoup4
!pip install requests

In [2]:
url ='https://en.wikipedia.org/wiki/World_Happiness_Report'

In [88]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')


def define_title(soup, tag, class_type, class_value):
    scrambled_titles_list = []
    scrambled_titles = soup.find_all(tag, {class_type : class_value})

    for scrambled_title in scrambled_titles:
        scrambled_titles_list.append(scrambled_title)
        
    return scrambled_titles_list
    
td_class_titles = define_title(soup, 'span', "class", "mw-headline")

def removed_tag_titles(soups=soup, tag= '', regex = ''):
    import re

    title_list = []

    for soup in soups:
        text_corpus = soup.find_all(tag) 
        if tag == '':
            text_corpus = soups
        if type(regex) == str:
            regex_in = regex
        elif type(regex) == list:
            regex_in = '|'.join(regex)
        else:
            raise ValueError('Use only a regex string or list of regex strings')

        for node in text_corpus:
            text = node.get_text()
            text= re.sub(regex_in, '', text)
            title_list.append(text)
            
    return title_list

filtered_titles = removed_tag_titles(td_class_titles, '' , ["\[[^\]]*\]", '\n'])


def make_titles(arr):
    final_titles = {}
    j = 0
    for i, val in enumerate(arr, 1):
        if val != '\n':
            j += 1
            final_titles[f'Table_{j}'] = val
    return final_titles

final_titles = make_titles(filtered_titles)



def locate_tables(soup=soup):
    tbody_class_tables_dict = {}
    wikitable_sortable_class = soup.find_all("table", {"class": "wikitable sortable"})
    for i, wiki_table in enumerate(wikitable_sortable_class):
        tbody_class_tables_dict[f'Table_{i + 1}'] = wikitable_sortable_class[i]
            
    return tbody_class_tables_dict

tbody_class_tables_dict = locate_tables(soup)

def removed_tag_tables(soup=soup, tag= '', regex = ''):
    import re
    text_corpus = soup.find_all(tag) 
    final_text = []
    
    if type(regex) == str:
        regex_in = regex
    elif type(regex) == list:
        regex_in = '|'.join(regex)
    else:
        raise ValueError('Use only a regex string or list of regex strings')

    for node in text_corpus:
        text = node.get_text()
        text= re.sub(regex_in, '', text)
        final_text.append(text)
    return final_text

def filter_tables(tables_dict):        
    filtered_table_dict = {}
    for key, val in tables_dict.items():

        tbody_class_table_clean = removed_tag_tables(val, 'tr' , ["\[[^\]]*\]", '\xa0', ',', '\n$', '^\n'])
        filtered_table_dict[key] = tbody_class_table_clean
        
    return filtered_table_dict

filter_tables_dict = filter_tables(tbody_class_tables_dict)


def fill_nulls(tables_dict):
    import numpy as np
    filled_filter_tables_dict = {}
    for key, table in tables_dict.items():
        filled_nulls_table = []
        for i, row in enumerate(table):
            if '\n' in row[0:1]:
                row_filled = 'NaN' + row           
                filled_nulls_table.append(row_filled)
            else:
                row_filled = row
                filled_nulls_table.append(row_filled)
        filled_filter_tables_dict[key] = filled_nulls_table
    return filled_filter_tables_dict
    
filtered_tables_rows = fill_nulls(filter_tables_dict)

def make_tables(tables_dict):
    import re   
    final_tables_dict = {}
    for key, table in tables_dict.items():
        filtered_tables_rows_dict = {}
        for i, val in enumerate(table, 1):
            row = re.split(r"\n", val)
            while "" in row:
                row.remove("")
            filtered_tables_rows_dict[f'Row_{i}'] = row

        final_tables_dict[key] = filtered_tables_rows_dict
    return final_tables_dict
    
final_rows = make_tables(filtered_tables_rows)

#tables_dict = {key: tables{key: rows[values]}}

def make_dataframe(tables_dict):
    tables_to_df = {}
    for key, table in tables_dict.items():
        same = True
        
        for i in range(len(table.values()) - 1):
            length = len(list(table.values())[i])
            if len(list(table.values())[i+1]) == length:
#                 print(f'good: {list(table.values())[i], list(table.values())[i+1]}')
                pass
            else:
#                 print(f'bad: {list(table.values())[i], list(table.values())[i+1]}')
                same = False
                        
        if same == True:
            tables_to_df[key] = table
        else:
            print(f'{key} could not be added, the length of the rows are not the same')
            same = True
    
    final_tables = {}

    
    for key, table in tables_to_df.items():
        final_tables[key] = pd.DataFrame.from_dict(table).transpose()
        final_tables[key] = final_tables.get(key).reset_index(drop = True)
        final_tables[key].columns = final_tables[key].iloc[0]
        final_tables[key] = final_tables[key][1:]
    return final_tables

final_tables = make_dataframe(final_rows)

def name_tables(titles, tables):

    final_df = {titles[key] : value for key, value in tables.items()} 
    return final_df


final_df = name_tables(final_titles, final_tables)

def finaldf_to_csv(final_df):
    for i, table in enumerate(final_df): 
        final_df[table].to_csv('GDP(Nominal)'+str(table)+'.csv')
        print(f'Table_{i + 1} was saved in CSV format!')
finaldf_to_csv(final_df)



Table_4 could not be added, the length of the rows are not the same
Table_5 could not be added, the length of the rows are not the same
{'Table_1': {'Row_1': ['Overall rank', 'Country or region', 'Score', 'GDP per capita', 'Social support', 'Healthy life expectancy', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption'], 'Row_2': ['1', 'Finland', '7.809', '1.285', '1.500', '0.961', '0.662', '0.160', '0.478'], 'Row_3': ['2', 'Denmark', '7.646', '1.327', '1.503', '0.979', '0.665', '0.243', '0.495'], 'Row_4': ['3', 'Switzerland', '7.560', '1.391', '1.472', '1.041', '0.629', '0.269', '0.408'], 'Row_5': ['4', 'Iceland', '7.504', '1.327', '1.548', '1.001', '0.662', '0.362', '0.145'], 'Row_6': ['5', 'Norway', '7.488', '1.424', '1.495', '1.008', '0.670', '0.288', '0.434'], 'Row_7': ['6', 'Netherlands', '7.449', '1.339', '1.464', '0.976', '0.614', '0.336', '0.369'], 'Row_8': ['7', 'Sweden', '7.353', '1.322', '1.433', '0.986', '0.650', '0.273', '0.442'], 'Row_9': ['8', 'New 